In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import sys

In [ ]:
import os

In [ ]:
#basedir = "/home/marco/git/webvalley/datapreproc"
#basedir = "C:/Users/julix/webvalley_git/full_project/data/PLIC-Milano"
basedir = "/home/mattia/Scrivania/plic_clinical_data/plic_clinical_data/PLIC-Milano"

# Import the smallest dataset

The plic-milano-foglio-piccolo.xlsx is PLIC_1445_V1_V2_V3_V4_linear variables _ updated 15_01_19.xlsx
and the plic-milano-foglio-grande.xlsx is PLIC_dataset parametri clinici_V1_V2_V3_V4_1445.xlsx

In [ ]:
smalldf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-piccolo.xlsx"), index_col=0).fillna(-1)

In [ ]:
smalldf.head()

In [ ]:
smalldf.info()

# What is male and what is female?
### Should at least be written in the docs of the whole software otherwise it will be impossible to import new data

In [ ]:
smalldf.sesso.unique()

# Import the biggest dataset

In [ ]:
bigdf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-grande.xlsx"), index_col=0).fillna(-1)

In [ ]:
bigdf.info()

In [ ]:
bigdf.head()

# At least one column is not in the biggest dataset

In [ ]:
smalldfcols = set(smalldf.columns.values)

In [ ]:
bigdfcols = set(bigdf.columns.values)

In [ ]:
f1_cols_in_f2 = smalldfcols.intersection(bigdfcols)

In [ ]:
f1_cols_non_in_f2 = smalldfcols.difference(bigdfcols)

In [ ]:
f1_cols_in_f2

In [ ]:
f1_cols_non_in_f2

In [ ]:
len(f1_cols_in_f2)

In [ ]:
len(f1_cols_non_in_f2)

Checking if column names differ because of case sensitivity

In [ ]:
bigdfcols_lower = set([k.lower() for k in bigdfcols])

In [ ]:
smalldfcols_lower = set([k.lower() for k in smalldfcols])

In [ ]:
len(smalldfcols_lower.intersection(bigdfcols_lower))

which is not the case

# Check how many patients in the smaller dataset are also in the biggest one

In [ ]:
paz_in_smalldf = set(smalldf.index.values)

In [ ]:
paz_in_bigdf = set(bigdf.index.values)

In [ ]:
paz_in_smalldf.difference(paz_in_bigdf)

In [ ]:
paz_in_bigdf.difference(paz_in_smalldf)

*All the patients in the small dataset are also in the biggest one*

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()

# Fix known issues with the dataset

The two datasets uses different ways to store the data (M/F -> 0/1; Yes/No -> 1/0)

In [ ]:
smalldf["sesso"] = smalldf["sesso"].map({1: "F", 0: "M"})

*We dropped all sex errors so the method is appropriate; now replace even Si and No*

In [ ]:
# Check all the columns that contains at least one Si or No

bool_cols = list()

for col in f1_cols_in_f2:
    k = getattr(bigdf, col).values
    if np.isin("Sì", k) or np.isin("No", k):
        bool_cols.append(col)

In [ ]:
bool_cols

In [ ]:
# Replace em all

for col in bool_cols:
    smalldf[col] = smalldf[col].map({0: "No", 1: "Sì"})

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()


## Now check if dataset 1 and dataset 2 shared patients contains same data

In [ ]:
# Pick every patient in the big dataset and check if the values are the same
# that are stored in the smaller one

for paz_to_be_checked in paz_in_smalldf:
    paz_from_big = bigdf.loc[paz_to_be_checked]
    paz_from_small = smalldf.loc[paz_to_be_checked]
    if not np.array_equal(paz_from_big[f1_cols_in_f2].values, paz_from_small[f1_cols_in_f2].values):
        print("Paz %s has differences" % paz_to_be_checked)
print("check complete")

# Adding non-duplicate data from smaller dataframe to bigger dataframe

## Preparing smaller dataframe to join with bigger dataframe

In [ ]:
smalldf.info()

In [ ]:
smalldf.drop(f1_cols_in_f2, axis=1, inplace=True)

In [ ]:
smalldf.info()

In [ ]:
bigdf.info()

In [ ]:
joined_df = bigdf.join(smalldf, how='outer')

In [ ]:
joined_df.info()
joined_df['menarca_1'].unique()

# Translate variable names

In [ ]:

trsl = [] #words to translate
suff = [] #suffixes -> other words that do not need to be translated (e.g. numbers or strings that are not actual words)

#divide suffixes from words to translate
for b in joined_df.columns:
    a = b.lower().split("_")
    c = 0
    slist = []
    for i in range(len(a),0,-1):
        c += 1
        if c == 3:
            break
        else:
            if len(a[-1]) <= 2:     #filter for non-words TODO: improve filter
                slist.append(a[-1])
                a.pop(-1)
    slist = slist[::-1]
    suff.append(slist)
    trsl.append(" ".join(a))
    
s = ",".join(trsl).lower() #string containing italian labels

#*copy and paste from google translate the translations of the labels* 
# NB. google translate accepts only 5000 characters at time, so you'll have to copy and paste several times 

s = " visits, standardized visits, death, death, death, date of visit, date of visit, date of visit, date of visit, sex, birth, age, age, age, age, education, education, education, physiological history, menarche, menarche, menarche, menarche, pregnancies, pregnancies, pregnancies, abortion, pregnancies carried to term, menopause, induced, menopause, induced, menopause, induced, menopause age, induced, alcohol, b wine, b beer, b spirits, alcohol , b wine, b beer, b spirits, alcohol, b wine, b beer, b spirits, alcohol, b wine, b beer, b spirits, smoke, smoke 1 recod, smoke, smoke 2 recod, smoke, smoke 3 recod, smoke, smoke 4 recod, physical activity, physical activity intensity, physical activity hours, physical activity, physical activity intensity, physical activity hours, physical activity, physical activity intensity, physical activity hours, physical activity, physical activity intensity, physical activity hours, pathological history, chd, chd, chd, chd, chd profile, remote chd, remote chd, remote chd, remote chd, angina, angina date, angina, date angina, angina, date angina, angina, date angina, profile angina, remote angina, remote angina, remote angina, remote angina, silent ischemia, data silent ischemia, silent ischemia, silent ischemia date, silent ischemia, ischemia silent data, silent ischemia, given silent ischemia, silent profile ischemia, remote silent ischemia, remote silent ischemia, remote silent ischemia, remote silent ischemia, ima, ima date, ima, ima date, ima date, ima, ima date, ima date profile, remote ima, remote ima, remote ima, remote ima, bypass, data bypass, bypass, data bypass, bypass, data bypass, bypass, data bypass, profile bypass, remote bypass, remote bypass, remote bypass, remote bypass, ptca, ptca date, ptca, ptca date, ptca, ptca date, ptca, ptca date, ptca profile, remote ptca, remote ptca, remote ptca, remote ptca, stroke, date stroke, stroke, stroke date, stroke, date stroke, stroke, stroke date, stroke profile, remote stroke, remote stroke, remote stroke, remote stroke, tia, date tia, tia, date tia, tia, date tia, tia, date tia, tia profile, tia remote, tia remote, remote thia, remote thia, peripheral arterial disease, given peripheral arterial disease, peripheral arteriopathy, given peripheral arterial disease, peripheral arterial disease, given peripheral arterial disease, tsa data peripheral arteriopathy, peripheral arterial disease, peripheral arterial disease , remote peripheral arterial disease, remote peripheral artery disease, remote peripheral artery disease, aortic artery disease, aortic artery disease, aortic artery disease, aortic artery disease, aortic artery disease, aortic artery disease, aortic artery disease, artery disease, aortic artery disease remote aortic, remote aortic arteriopathy, atrial fibrillation, given atrial fibrillation, atrial fibrillation, given atrial fibrillation, atrial fibrillation, given atrial fibrillation, atrial fibrillation, given atrial fibrillation, arrhythmia, given arrhythmia, arrhythmia given, Arith ie major, major arrhythmias data, ivs, ivs, ivs, ivs data, ivs, ivs date, decompensation, decompensation given, decompensation, decompensation given, decompensation, decompensation given, decompensation, decompensation given, remote decompensation, remote decompensation, remote decompensation, remote decompensation, venous thrombosis, given venous thrombosis, venous thrombosis, venous thrombosis, venous thrombosis, given venous thrombosis, venous thrombosis, venous thrombosis, cerebral haemorrhage, given cerebral hemorrhage, cerebral hemorrhage, cerebral hemorrhage, cerebral hemorrhage data, cerebral haemorrhage, given cerebral hemorrhage, self hypertension, self data hypertension, self hypertension, self data hypertension, self hypertension, self data hypertension, self hypertension, self data hypertension, self diabetes, self diabetes1 self data , diabetes1 self, diabetes1 self data, diabetes1 self, diabetes1 self data, diabetes2 self, diabetes2 self data, diabetes2 self, diabetes2 self data, diabetes2 self, diabetes2 self data, diabetes2 self, diabetes2 self data, obesity self, self data obesity, self obesity, self data obesity, self obesity, self data obesity, self obesity, self data obesity, self dyslipidemia, self data dyslipidemia, self data dyslipidemia, self data dyslipidemia, self dyslipidemia , self type dyslipidemia, self data dyslipidemia, new revaluation diagnosis, new revaluation diagnosis, xanthomatosis, given xanthomatosis, xanthomatosis, given xanthomatosis, xanthomatosis, given xanthomatosis, xanthomatosis, given xanthomatosis, nephropathy, nephropathy, given nephropathy, nephropathy, type nephropathy, nephropathy given, nephropathy, nephropathy type, nephropathy given, nephropathy, nephropathy type, nephropathy given, nephropathy profile, thyroid disease, thyroid function, thyroid disease, thyroid disease, thyroid function, thyroid disease, thyroid function , thyroid disease, thyroid disease, thyroid function, thyroid disease, liver disease, liver disease, given liver disease, hepatopathic diseases and, type hepatopathies"

s = s.replace(" and","e").replace("given","date").replace("data","date").replace("b ","drinks ") #apply some corrections

s = [a[1:] for a in s.split(",")] #remove space before word

#reattach the suffixes
i = 0
for sub in s:
    sub = sub.replace(" ","_") + "_" + "_".join(suff[i])
    print(sub)
    i += 1

#joined_df['arteriopatia_aortica_data_4'].unique()      
#[b.lower() for b in joined_df.columns]
#trsl

    

In [ ]:
#e1 = examination 1
#anamnesi fisiologica = physiological_medical_history
#mearca = 
#NB. What is istruzione?
#
new_labels = ["carried_out_examinations",
              "a_standardized_carried_out_examinations",
              "death_e1_e2", #ask
              "death_e2_e3",
              "death_e3_e4",
              "e1_date",
              "e2_date",
              "e3_date",
              "e4_date",
              "gender",
              "birth_date",
              "e1_age",
              "e2_age",
              "e3_age",
              "e4_age",
              "e1_education",
              "e2_education",
              "e3_education",
              "e4_education"
              "physiological_medical_history",
              "e1_first_menstruation_age",
              "e2_first_menstruation_age",
              "e3_first_menstruation_age",
              "e4_first_menstruation_age",
              "e1_n_pregnancies",
              "e2_n_pregnancies",
              "e3_n_pregnancies",
              "e4_n_pregnancies",
              "e4_n_abortion",#why only e4?
              "e4_n_completed_pregnancies",
              "e1_menopause_age",
              "e1_menopause_induced",
              "e2_menopause_age",
              "e2_menopause_induced",
              "e3_menopause_age",
              "e3_menopause_induced",
              "e4_menopause_age",
              "e4_menopause_induced",
              "e1_alcohol_consumption",
              "e1_wine",
              "e1_beer",
              "e1_super_alcoholic"]
              
              
              
              
              
print("The end.")